In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import json

import os
import base64

driver = webdriver.Firefox()
path = "https://eumostwanted.eu/"

In [14]:
driver.get(path)

In [15]:
from selenium.common.exceptions import NoSuchElementException
#Ajustmaos el driver a pantalla completa
driver.maximize_window()


try:
    elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/div[3]/div[2]/main/div[2]/div/div[1]/div/div/div/div[1]/div/div/span")
    back = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[3]/div[2]/main/div[2]/div/div[1]/div/div/div/div[1]/div[1]")

    lista_buscaos = []
    lista_json = []

    for element in elements:
        buscado = element.text.replace(",","").replace(" ","-").lower()
        lista_buscaos.append(buscado)


    i = 0
    for i in range(len(elements)):
        element = driver.find_element(By.XPATH, f"/html/body/div[1]/div/div/div[3]/div[2]/main/div[2]/div/div[1]/div/div/div/div[1]/div[{i+2}]")

        # Si el elemento tiene la clase arrested, lo ignoramos
        if ("arrested" in element.get_attribute("class") or "separator" in element.get_attribute("class")):
            continue

        driver.execute_script("arguments[0].click();", element)
        time.sleep(1)

        wanted_image = driver.find_element(By.CLASS_NAME, "show")

        #Accedemos a la etiqueta img de wanted_image
        wanted_image = wanted_image.find_element(By.TAG_NAME, "img")
        wanted_image = wanted_image.get_attribute("src")

        #Hacemos un get a la imagen y la guardamos como base64 en la carpeta imagenes2
        response = requests.get(wanted_image)
        
        #Convertimos la imagen a base64
        image_data = base64.b64encode(response.content).decode('utf-8')
        
        wanted_data_label = driver.find_elements(By.CLASS_NAME, "field-label")

        # Creamos un json con los datos de la persona
        data = {}
        for x in range(len(wanted_data_label)):
            value = wanted_data_label[x].find_element(By.XPATH, "following-sibling::*[1]")
            data[wanted_data_label[x].text] = value.text


        # Añadimos la imagen al json
        data["imagen"] = image_data

        lista_json.append(data)

        driver.save_screenshot(fr'C:\Users\alejandro.regueiroru\Desktop\CursoIA\Big_Data\Request\Ejercicio_Europol\Imagenes\{lista_buscaos[i]}.png')
        back.click()
        time.sleep(0.25)

      # Guardamos en un json quien es y donde trabaja
    with open(fr'C:\Users\alejandro.regueiroru\Desktop\CursoIA\Big_Data\Request\Ejercicio_Europol\Json\buscados.json', 'w', encoding='utf-8') as f:
        json.dump(lista_json, f, indent=4)

    driver.quit()

except NoSuchElementException as e:
    print(e)